In [1]:
#!pip install datasets
#!pip install --upgrade --force-reinstall huggingface_hub

In [2]:
import torch
import torchvision
from torchvision.transforms import transforms, Lambda, Resize
from torchvision import datasets, transforms, models
import os
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Specify where to find the data preparation class
import sys
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [3]:
# ResNet50 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
#DIMENSIONS = 3
#SIZE = 256

LR = 0.1
MOMENTUM=0.85
WEIGHT_DECAY = 1e-4
LR_STEP_SIZE = 5
LR_GAMMA = 0.1

In [4]:
dataloaders = {x: CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type=x, mean=MEAN, std=STD, dimensions=3).data_loader for x in ['train', 'test']}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'test']}
print(dataset_sizes)

class_names = dataloaders['train'].dataset.classes
print(class_names)

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(dataloaders['train']):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

{'train': 1753, 'test': 449}
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([6, 4, 3, 0, 0])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 3, 3, 4, 5])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([5, 0, 6, 6, 0])


In [5]:
dataloaders['test']

In [6]:
# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP_SIZE, gamma=LR_GAMMA)


# Move the model to the GPU if available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# num_epochs = 20
# for epoch in tqdm(range(num_epochs)):
#     for phase in ['train', 'test']:
#         if phase == 'train':
#             model.train()
#         else:
#             model.eval()

#         running_loss = 0.0
#         running_corrects = 0

#         for inputs, labels in dataloaders[phase]:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             if phase == 'train':

#                 optimizer.zero_grad()

#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)

#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             LR_SCHEDULER.step()

#             elif phase == 'test':
#                 with torch.inference_mode():  
#                     running_loss = 0
#                     for step, (images, labels) in enumerate(dataloaders[phase]):
#                         images, labels = images.to(device), labels.to(device)
                        
#                         outputs = model(images)
#                         loss = criterion(outputs, labels)

#                         running_loss += loss.item()

#     epoch_loss = running_loss / dataset_sizes[phase]
#     epoch_acc = running_corrects.double() / dataset_sizes[phase]

#     print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
# print("Training complete!")

In [8]:
num_epochs = 25

print("[INFO] Start training")
print("---------------------")

for epoch in tqdm(range(num_epochs)):

    total_batch = len(dataloaders['train'].dataset)//16
    
    # Training phase
    model.train()  # Set the model to train mode

    # Get statistics
    epoch_loss = 0
    len_dataset = 0
    
    for step, (batch_images, batch_labels) in enumerate(dataloaders['train']):
        X, Y = batch_images.to(device), batch_labels.to(device)

        pred = model(X)
        loss = criterion(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += pred.shape[0] * loss.item()
        len_dataset += pred.shape[0]
        if (step) % 10 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, step+1, total_batch, loss.item()))

    epoch_loss = epoch_loss/ len_dataset
    print('Epoch: ', epoch+1, '| train loss : %0.4f' % epoch_loss)

    LR_SCHEDULER.step()
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    with torch.inference_mode():  
        running_loss = 0
        for step, (images, labels) in enumerate(dataloaders['test']):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            
    running_loss = running_loss / len(dataloaders['test'])
    print('Epoch: ', epoch, '| test loss : %0.4f' % running_loss )

[INFO] Start training
---------------------


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], lter [1/1752] Loss: 8.3915
Epoch [1/25], lter [11/1752] Loss: 43.5536
Epoch [1/25], lter [21/1752] Loss: 48.0594
Epoch [1/25], lter [31/1752] Loss: 30.8403
Epoch [1/25], lter [41/1752] Loss: 11.7199
Epoch [1/25], lter [51/1752] Loss: 55.1133
Epoch [1/25], lter [61/1752] Loss: 30.1797
Epoch [1/25], lter [71/1752] Loss: 43.7683
Epoch [1/25], lter [81/1752] Loss: 37.7293
Epoch [1/25], lter [91/1752] Loss: 11.4472
Epoch [1/25], lter [101/1752] Loss: 23.5558
Epoch [1/25], lter [111/1752] Loss: 17.9113
Epoch [1/25], lter [121/1752] Loss: 27.5843
Epoch [1/25], lter [131/1752] Loss: 16.4878
Epoch [1/25], lter [141/1752] Loss: 11.9479
Epoch [1/25], lter [151/1752] Loss: 31.8886
Epoch [1/25], lter [161/1752] Loss: 26.3392
Epoch [1/25], lter [171/1752] Loss: 26.2997
Epoch [1/25], lter [181/1752] Loss: 21.3404
Epoch [1/25], lter [191/1752] Loss: 20.3694
Epoch [1/25], lter [201/1752] Loss: 20.1641
Epoch [1/25], lter [211/1752] Loss: 10.2102
Epoch [1/25], lter [221/1752] Loss: 19.1473


  4%|▍         | 1/25 [27:46<11:06:29, 1666.24s/it]

Epoch:  0 | test loss : 13.5076
Epoch [2/25], lter [1/1752] Loss: 9.6151
Epoch [2/25], lter [11/1752] Loss: 17.8819
Epoch [2/25], lter [21/1752] Loss: 36.2848
Epoch [2/25], lter [31/1752] Loss: 17.3159
Epoch [2/25], lter [41/1752] Loss: 39.4571
Epoch [2/25], lter [51/1752] Loss: 5.9503
Epoch [2/25], lter [61/1752] Loss: 21.7455
Epoch [2/25], lter [71/1752] Loss: 16.6650
Epoch [2/25], lter [81/1752] Loss: 12.3730
Epoch [2/25], lter [91/1752] Loss: 21.7604
Epoch [2/25], lter [101/1752] Loss: 21.8858
Epoch [2/25], lter [111/1752] Loss: 23.3653
Epoch [2/25], lter [121/1752] Loss: 19.2023
Epoch [2/25], lter [131/1752] Loss: 17.1474
Epoch [2/25], lter [141/1752] Loss: 17.0753
Epoch [2/25], lter [151/1752] Loss: 9.1430
Epoch [2/25], lter [161/1752] Loss: 9.0247
Epoch [2/25], lter [171/1752] Loss: 15.6967
Epoch [2/25], lter [181/1752] Loss: 16.7071
Epoch [2/25], lter [191/1752] Loss: 21.3785
Epoch [2/25], lter [201/1752] Loss: 9.9391
Epoch [2/25], lter [211/1752] Loss: 24.6614
Epoch [2/25], lt

...

Epoch:  22 | test loss : 2.0318
Epoch [24/25], lter [1/1752] Loss: 1.3431
Epoch [24/25], lter [11/1752] Loss: 1.7369
Epoch [24/25], lter [21/1752] Loss: 2.4375
Epoch [24/25], lter [31/1752] Loss: 1.7923
Epoch [24/25], lter [41/1752] Loss: 1.8515
Epoch [24/25], lter [51/1752] Loss: 2.4525
Epoch [24/25], lter [61/1752] Loss: 2.2459
Epoch [24/25], lter [71/1752] Loss: 2.4314
Epoch [24/25], lter [81/1752] Loss: 1.6011
Epoch [24/25], lter [91/1752] Loss: 1.7558
Epoch [24/25], lter [101/1752] Loss: 2.2122
Epoch [24/25], lter [111/1752] Loss: 1.5766
Epoch [24/25], lter [121/1752] Loss: 1.1077
Epoch [24/25], lter [131/1752] Loss: 1.3303
Epoch [24/25], lter [141/1752] Loss: 2.7778
Epoch [24/25], lter [151/1752] Loss: 2.6302
Epoch [24/25], lter [161/1752] Loss: 1.9314
Epoch [24/25], lter [171/1752] Loss: 1.7987
Epoch [24/25], lter [181/1752] Loss: 1.0478
Epoch [24/25], lter [191/1752] Loss: 1.3958
Epoch [24/25], lter [201/1752] Loss: 1.0508
Epoch [24/25], lter [211/1752] Loss: 1.6010
Epoch [24/2

 96%|█████████▌| 24/25 [11:05:15<27:43, 1663.78s/it]

Epoch:  23 | test loss : 2.0045
Epoch [25/25], lter [1/1752] Loss: 1.4326
Epoch [25/25], lter [11/1752] Loss: 1.2760
Epoch [25/25], lter [21/1752] Loss: 2.5940
Epoch [25/25], lter [31/1752] Loss: 2.2064
Epoch [25/25], lter [41/1752] Loss: 1.7602
Epoch [25/25], lter [51/1752] Loss: 1.3228
Epoch [25/25], lter [61/1752] Loss: 1.5645
Epoch [25/25], lter [71/1752] Loss: 2.0609
Epoch [25/25], lter [81/1752] Loss: 0.9280
Epoch [25/25], lter [91/1752] Loss: 1.7571
Epoch [25/25], lter [101/1752] Loss: 1.9965
Epoch [25/25], lter [111/1752] Loss: 1.3140
Epoch [25/25], lter [121/1752] Loss: 2.0486
Epoch [25/25], lter [131/1752] Loss: 2.4989
Epoch [25/25], lter [141/1752] Loss: 1.9747
Epoch [25/25], lter [151/1752] Loss: 1.4622
Epoch [25/25], lter [161/1752] Loss: 2.0292
Epoch [25/25], lter [171/1752] Loss: 1.9456
Epoch [25/25], lter [181/1752] Loss: 1.7801
Epoch [25/25], lter [191/1752] Loss: 1.9872
Epoch [25/25], lter [201/1752] Loss: 1.7041
Epoch [25/25], lter [211/1752] Loss: 1.2126
Epoch [25/2

100%|██████████| 25/25 [11:32:55<00:00, 1663.04s/it]

Epoch:  24 | test loss : 1.9970


In [9]:
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['train']:
    # Forward pass
    inputs, target = batch_data
    output = model(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 46.58%


In [10]:
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['test']:
    # Forward pass
    inputs, target = batch_data
    output = model(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 42.51%


In [11]:
# Save the model
torch.save(model.state_dict(), 'ResNet50_v2.pth')